In [1]:
import torch

In [2]:
sigma = torch.Tensor([0.2])

def RBF_kernel(x,x_,dim):
    x = x.view(-1,1,dim)
    x_ = x_.view(1,-1,dim)
    dist = (x-x_).pow(2).sum(dim=-1)
    return torch.exp(-dist/2/sigma.pow(2))

In [35]:
class GaussianProcess:
    
    def __init__(self,dim_x,dim_y,var,kernel=None):
        self.dim_x = dim_x
        self.dim_y = dim_y
        self.var = var
        if kernel:
            self.kernel = kernel
        else:
            self.kernel = RBF_kernel
            
    def train(self,x,y):
            self.train_size = x.size()[0]
            self.train_x = x
            self.train_y = y
            self.Knn = self.kernel(x,x,self.dim_x)
            self.In = torch.eye(self.train_size)
            self.G = (self.Knn + self.var*self.In).inverse()
            
            
    def predict_(self,pseudo_x,pred_x):
            Kmm = self.kernel(pseudo_x,pseudo_x,self.dim_x)
            Kmm_inv = Kmm.inverse()
            Knm = self.kernel(self.train_x,pseudo_x,self.dim_x)
            Kmn = Knm.t()
            Kpm = self.kernel(pred_x,pseudo_x,self.dim_x)
            Kmp = Kpm.t()
            
            _L =  self.Knn - Knm@Kmm_inv@Kmn
            L = _L*I
            print(L)
            
            F = (L + self.var*self.In).inverse()
            
            Q = Kmm + Kmn@F@Knm
            Q_inv = Q.inverse()
            
            #print(Kpm.size(),Q_inv.size(),Kmn.size(),F.size(),self.train_y.size())
            
            myu_p = Kpm@Q_inv@Kmn@F@self.train_y
                    
            return myu_p

    
    def predict(self,pred_x):
        Kpp = self.kernel(pred_x,pred_x,self.dim_x)
        Kpn = self.kernel(pred_x,self.train_x,self.dim_x)
        Knp = Kpn.t()
        
        myu_p = Kpn@self.G@self.train_y
        sigma_p = Kpp - Kpn@self.G@Knp
        
        return myu_p,sigma_p
        
    
    def distance_measure(self):
        Kxn = self.kernel(x,self.train_x)
        Knx = Kxn.t()
        Kxx = self.kernel(x,self.train_x)
        return Kxx - Kxn@self.Knn_inv@Knx
    


In [16]:
class A:
    def __init__(self,b):
        a=3
        self.b=b
    def func(self):
        return a,b
    
A_ = A(2)
A_.func()

NameError: name 'a' is not defined

In [19]:

import math
import torch
import gpytorch
from matplotlib import pyplot as plt

In [20]:
# Training data is 11 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 10)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2

In [21]:
grid_x = torch.cat([torch.Tensor([a,b]).view(1,2) for a in train_x for b in train_x] ,dim=0  )
grid_y = torch.cat([torch.cat( [(a+b).view(1,1),(a*b).view(1,1)],dim=1) for a in train_x for b in train_x] ,dim=0  )

In [22]:
pseudo_x = torch.cat([torch.Tensor([a,b]).view(1,2) for a,b in zip(train_x,train_x)] ,dim=0  )
test_x = pseudo_x

In [23]:
pseudo_y  = torch.cat([torch.cat( [(a+b).view(1,1),(a*b).view(1,1)],dim=1) for a,b in zip(train_x,train_x)] ,dim=0  )
pseudo_y

tensor([[0.0000, 0.0000],
        [0.2222, 0.0123],
        [0.4444, 0.0494],
        [0.6667, 0.1111],
        [0.8889, 0.1975],
        [1.1111, 0.3086],
        [1.3333, 0.4444],
        [1.5556, 0.6049],
        [1.7778, 0.7901],
        [2.0000, 1.0000]])

In [30]:
dim_x = 2
dim_y = 2
var = 0.1
sgp = SparseGaussianProcess(dim_x,dim_y,var)
sgp.train(grid_x,grid_y)
sgp.predict_(test_x)




(tensor([[ 1.6677e-02, -9.0348e-04],
         [ 2.1537e-01,  1.1646e-02],
         [ 4.4378e-01,  4.9612e-02],
         [ 6.5938e-01,  1.1058e-01],
         [ 8.8602e-01,  1.9647e-01],
         [ 1.1137e+00,  3.1031e-01],
         [ 1.3126e+00,  4.3717e-01],
         [ 1.5338e+00,  5.9462e-01],
         [ 1.8455e+00,  8.2673e-01],
         [ 1.8585e+00,  9.1999e-01]]),
 tensor([[ 6.0444e-02,  2.5160e-03, -2.3574e-03,  1.3102e-03, -5.7346e-04,
           1.9375e-04, -3.8712e-05, -3.6486e-06,  1.7500e-05, -3.2173e-06],
         [ 2.5160e-03,  3.0043e-02,  9.1201e-03, -3.0400e-03,  8.6908e-04,
          -1.4166e-04, -4.1396e-05,  5.2130e-05, -4.1865e-05,  1.7500e-05],
         [-2.3578e-03,  9.1194e-03,  2.6938e-02,  8.6989e-03, -2.5032e-03,
           7.1120e-04, -1.4443e-04, -7.0003e-06,  5.2124e-05, -3.6450e-06],
         [ 1.3104e-03, -3.0401e-03,  8.6988e-03,  2.6539e-02,  8.8351e-03,
          -2.5496e-03,  7.4777e-04, -1.4438e-04, -4.1408e-05, -3.8700e-05],
         [-5.7343e-04,  